# AIML Coursework marker

In [1]:
import re
import aiml

def preprocessSingleInput(bot,theInput):
    # run the input through the 'normal' subber- only wortks for a single sentence
    subbed1 = bot._subbers['normal'].sub(theInput).upper()
    subbed2 = re.sub(bot._brain._puncStripRE, " ", subbed1)
    return(subbed2)
    

In [2]:
debug = True
debug2 = False
DEWIS_MSG_PREFIX = ""
DEWIS_SCORE_PREFIX = "DEWIS_SCORE"
theAIMLfile = '45cats.aiml'
theQuestionsFile = "coursework-questions-and-responses.txt"
outputFile = theAIMLfile[:-5] +"-results.txt"
NUMQS =45
contextQuestions = [32,45,44]

In [3]:
#declare arrays to hold the questions and answers
questions = []
responses = []

# read the questions and answers in
# Using readline() 
file1 = open(theQuestionsFile, 'r') 
thisQ = 0
  
while True: 
    # Get next line from file 
    line = file1.readline() 
    if not line: 
        print("unexpected end of file")
        break
    # should be a question
    elif (line[0] != 'Q' ):
        print("didn't get expected question marker Q")
        break;
    elif ( int(line[1:3]) != thisQ):
        print("question had wrong number")
        break
    else:
        questions.append( line[5:-1])
        if(debug2):
            print("question {} is: {}".format(thisQ,questions[thisQ]))        
        
    line = file1.readline() # next line should be the corresponding answer
    if not line: 
        print("unexpected end of file")
        break
    elif (line[0] != 'A' ):
        print("didn't get expected answer marker A")
        break;
    elif ( int(line[1:3]) != thisQ):
        print("answer had wrong number")
        break
    else:
        responses.append(line[5:-1])
        if(debug2):
            print("response {} is: {}".format(thisQ,responses[thisQ]))
    
    thisQ += 1
    # then read the empty line separating QnA paits
    line = file1.readline()
    
    # if line is empty 
    # end of file is reached 
    if not line: 
        break
    if(debug2):
        print("")

file1.close() 




# check that there are the right number
if (thisQ <NUMQS ):
    print("error, only {} question-answer pairs read".format(thisQ))
elif (len(questions) < NUMQS or len(responses)<NUMQS):
    print("error, somehow the questions or responses have not all be saved")
    if(debug):
        print(" {} questions and {}responses read, thisQ = {}".format(len(questions),len(responses),thisQ))
else: 
    print('{} question-response pairs read for testing your bot'.format(thisQ))

45 question-response pairs read for testing your bot


In [4]:
# Create Chatbot and read the candidate AIML file
checkBot = aiml.Kernel()
checkBot.verbose(True)

In [5]:
checkBot.resetBrain()
checkBot.learn(theAIMLfile)

# How many categories were correctly read
numCategories = checkBot.numCategories()
print( " {}  After reading your file the bot has {} categories".format(DEWIS_MSG_PREFIX,numCategories))
print( " {}  The bot will overwrite categories with the same pattern, that and topic".format(DEWIS_MSG_PREFIX,numCategories))
print('{} this number should help you fix misformed categories if needed'.format(DEWIS_MSG_PREFIX))
print()

Loading 45cats-with-that.aiml...done (0.01 seconds)
   After reading your file the bot has 45 categories
   The bot will overwrite categories with the same pattern, that and topic
 this number should help you fix misformed categories if needed



In [6]:
# either figure out how to query the bot categories
## or open the student.aiml file and read it line by line looking for <srai> <set> <star/> and <that>
file2 = open(theAIMLfile,'r')
srai_count = 0
set_count = 0
wildcard_count=0
starslash_count=0
that_count = 0
condition_count= 0

#read through line by line coutning uise of AIML constructs
while(True):
    line = file2.readline()
    if not line:
        break
    if "<srai>" in line:
        srai_count += 1
    if "<set" in line: # just use start - they ar hopefullty defining a name for their variable
        set_count += 1
    if ("*" in line) or ("_" in line) or ("^" in line) or ("#" in line):
        wildcard_count +=1
    if "<star" in line: #just look for start of tag in case they used indexing
        starslash_count += 1
    if "<that" in line: #just look for start of tag in case they used indexing
        that_count +=1
    if "<that" in line: #just look for start of tag in case they used indexing
        condition_count +=1
file2.close       
        
print('{} You used <srai> {} times,   used wildcards  {} times, and used the matched values {} times,'.format(DEWIS_MSG_PREFIX,srai_count,wildcard_count,starslash_count))
print('{} You set values for {} variables and used <that> {} times'.format(DEWIS_MSG_PREFIX,set_count,that_count) )
print('{} You changed the bots response depending on variable values {} times'.format(DEWIS_MSG_PREFIX,condition_count) )

 You used <srai> 0 times,   used wildcards  0 times, and used the matched values 0 times,
 You set values for 0 variables and used <that> 3 times
 You changed the bots response depending on variable values 3 times


In [7]:


# initialise score
numCorrect = 0
numContextQsCorrect=0

file3 = open(outputFile,'w')

for thisQ in range (NUMQS):
    #get bot's response to question
    botResponse = checkBot.respond(questions[thisQ])
    file3.write('Q{:2d}: {}\n'.format(thisQ, questions[thisQ]))
    file3.write('Expected response: {}\n'.format(responses[thisQ]))
    file3.write('Your bot response: {}\n'.format(botResponse))
    # check if it matches the required input
    if botResponse == responses[thisQ] :
        #print('question {} answered correctly'.format(thisQ))
        file3.write('*** Question answered correctly\n\n')
        numCorrect +=1
        if thisQ in contextQuestions:
            numContextQsCorrect +=1
    else:
        file3.write('Question answered incorrectly\n\n')
        if(debug):
            theInput = questions[thisQ]
            print('Q{} {}\n gets preprocessed as:{}'.format(thisQ,theInput,preprocessSingleInput(checkBot,theInput)))
            print(' expected :' +responses[thisQ])
            print(' got      :' +botResponse)
            lastThat = checkBot.getPredicate("_outputHistory")

# write final line to log file and exit
file3.write(' In total you got {} questions correct'.format(numCorrect))
file3.close()

In [8]:
# calculate final score
finalScore= numCorrect 
# if all questions correct then we start rewarding go solutions
if (numCorrect==NUMQS):
    if(numCategories < 5):
        finalScore= 65
    elif (numCategories <10):
        finalScore = 100
    else:
        finalScore = 100 - numCategories

# provide output for DEWIS
print('{} {}'.format(DEWIS_SCORE_PREFIX,finalScore))

    

DEWIS_SCORE 55


In [9]:
print (numCorrect,numCategories)

45 45


In [10]:
#keepgoing= True
#while(keepgoing):
#    nextInput = input("Enter your message >> ")
#    if (nextInput=='bye'):
#       keepgoing= False
#    else:
#        print (checkBot.respond(nextInput))